# HW1 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Jan 25 2018

In [1]:
import numpy as np
from IPython.display import display, Math, Latex

def bmatrix(a):
    """Returns a LaTeX bmatrix
    https://stackoverflow.com/a/17131750/2577586

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

## 1. 

> Implement the $PA=LDU$ decomposition algorithm by yourself (i.e. do not just call a built-in function in Matlab or Python. You may assume the matrix $A$ is square and of full rank. Show that your implementation is functional.

See original jupyter notebook for code.

In [23]:
def plud(a):
    """
    Inspired by procedure presented in book chapter:
    http://www.math.iit.edu/~fass/477577_Chapter_7.pdf
    """
    shape = a.shape
#     print("a.shape: ", a.shape)
    p = np.identity(shape[0])
    l = np.identity(shape[0])
    p_s = []
    l_s = []
    a_i = a

    for i in range(shape[0]-1):
#         print("i: ", i)
#         print("a_i: \n", a_i)

        # Determine the next largest pivot by index
        pivot_search_space = a_i[i:,i]
        abs_pivot_search_space = np.absolute(pivot_search_space)
#         abs_pivot_search_space = pivot_search_space
        pivot_argmax = np.argmax(abs_pivot_search_space)
        max_pivot_index = pivot_argmax + i

#         print("max_pivot_index: ", max_pivot_index)

        # Create next permutation matrix
        p_i = np.identity(shape[0])
        p_i[[i, max_pivot_index],:] = p_i[[max_pivot_index, i],:]
#         print("p_i: \n", p_i)
    
        # Build up the eventual permutation 
        p = p_i.dot(p)
        p_s.append(p_i)

        # Apply current permutation to current iteration of A
        a_i = p_i.dot(a_i)
        
        # Create next L matrix
        l_i = np.identity(shape[0])
        
        # Calculate the row negations
        for j in range(i+1, shape[0]):
#             print("j: ", j)
            l_i[j, i] = - a_i[j, i] / a_i[i, i]
#         print("l_i: \n", l_i)

        # Build up the eventual L matrix 
        l = l_i.dot(l)
        l_s.append(l_i)
        
        # Apply current L matrix to current iteration of A
        a_i = l_i.dot(a_i)
#         print("a_i: \n", a_i, "\n\n")
    
    # Ivert L matrix for expected form
    l = np.linalg.inv(l)
    
    # Capture U matrix for simple PA = LU
    u = a_i
    
#     return p, l, u

    # Create D by extracting diagonal of U
    d = np.diag(np.diag(u))

    # Dormalize diagonal of U
    u /= np.diag(u)[:, None]
    
    return p, l, d, u

We'll use test this with the given A:

In [3]:
a = np.array([[4, 7, 0],
               [2, 2,-6],
               [3, 2, 1]])
display(Math('A=' + bmatrix(a)))

<IPython.core.display.Math object>

Then run our function to compute P, L D, and U matrix for the following form:

In [4]:
(P, L, D, U) = plud(a)

In [5]:
eq3 = "A=PLDU"
display(Math(eq3))

<IPython.core.display.Math object>

In [6]:
eq4 = 'P=' + bmatrix(P) + '\quad ' + \
      'L=' + bmatrix(L) + '\quad ' + \
      'D=' + bmatrix(D) + '\quad ' + \
      'U=' + bmatrix(U)
display(Math(eq4))

<IPython.core.display.Math object>

If we then plug the found matixs back into the eq above, we see that we get back the orignal A:

In [7]:
A2 = P.dot(L.dot(D.dot(U)))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

### Comparison
Now let us compare our implentation to the linuar algrabra library in [scipy.linalg.lu](https://docs.scipy.org/doc/scipy-0.15.0/reference/generated/scipy.linalg.lu.html#scipy.linalg.lu) using a previous [example](https://stackoverflow.com/a/45450574/2577586).

In [8]:
import numpy as np
import scipy.linalg as la

In [9]:
(P, L, U) = la.lu(a)

The scipy implementation returns only P, L and U for the form:

In [10]:
eq1 = "A=PLU"
display(Math(eq1))

<IPython.core.display.Math object>

In [11]:
eq2 = 'P=' + bmatrix(P) + '\quad ' + \
      'L=' + bmatrix(L) + '\quad ' + \
      'U=' + bmatrix(U)
display(Math(eq2))

<IPython.core.display.Math object>

We can recompute U and D by extracting the diagle along U for D, then normalizing U along the diagonal. 

In [12]:
D = np.diag(np.diag(U))
U /= np.diag(U)[:, None]

In [13]:
eq3 = "A=PLDU"
display(Math(eq3))

<IPython.core.display.Math object>

In [14]:
eq4 = 'P=' + bmatrix(P) + '\quad ' + \
      'L=' + bmatrix(L) + '\quad ' + \
      'D=' + bmatrix(D) + '\quad ' + \
      'U=' + bmatrix(U)
display(Math(eq4))

<IPython.core.display.Math object>

Agian, we get back the same A by re-multiplying the matrices as in the eq above.

In [15]:
A2 = P.dot(L.dot(D.dot(U)))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

## 2. 
> Compute the $PA=LDU$ decomposition and the SVD decomposition for each of the following matrices: (you can use your own LDU implementation and it is OK to use a pre-defined implementation for SVD).

Again, runing our custom $PA=LDU$ decomposition function on the following matrices, along with SVD, then reconstruction A from the returned matrices to note behavior.

### a.

In [16]:
a1 = np.array([[4, 7, 0],
               [2, 2,-6],
               [3, 2, 1]])
display(Math('A_1=' + bmatrix(a1)))

<IPython.core.display.Math object>

In [17]:
(P, L, D, U) = plud(a1)

eq4 = 'P=' + bmatrix(P) + '\quad ' + \
      'L=' + bmatrix(L) + '\quad ' + \
      'D=' + bmatrix(D) + '\quad ' + \
      'U=' + bmatrix(U)
display(Math(eq4))

A2 = P.dot(L.dot(D.dot(U)))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
(U, s, V) = np.linalg.svd(a1)

# full_matrices=True
# S = np.zeros(U.shape, dtype=complex)
# S[:U.shape[0], :U.shape[1]] = np.diag(s)

# full_matrices=False
S = np.diag(s)

eq4_svd = 'U=' + bmatrix(U) + '\quad \\ ' + \
          'S=' + bmatrix(S)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(V)
display(Math(eq4_svd))

A2 = np.dot(U, np.dot(S, V))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Here we'll note that none of the values in U, S, V are terribly small or large, so we are not too close too a singularity. Additinaly, our reconstruction of A is quite accurate, with some small rounding error for value 0 in A.

### b.

In [19]:
a2 = np.array([[1, 0, 0, 0, 1],
               [0, 1, 0, 1, 0],
               [0, 0, 1, 0, 0],
               [0, 1, 0, 0, 0],
               [1, 0, 0, 0, 0],])
display(Math('A_2=' + bmatrix(a2)))

<IPython.core.display.Math object>

In [22]:
(P, L, D, U) = plud(a2)

eq4 = 'P=' + bmatrix(P) + '\quad ' + \
      'L=' + bmatrix(L) + '\quad ' + \
      'D=' + bmatrix(D) + '\quad ' + \
      'U=' + bmatrix(U)
display(Math(eq4))

A2 = P.dot(L.dot(D.dot(U)))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [24]:
(U, s, V) = np.linalg.svd(a2, full_matrices=False)

# full_matrices=True
# S = np.zeros(U.shape, dtype=complex)
# S[:U.shape[0], :U.shape[1]] = np.diag(s)

# full_matrices=False
S = np.diag(s)

eq4_svd = 'U=' + bmatrix(U)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(S)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(V)
display(Math(eq4_svd))

A2 = np.dot(U, np.dot(S, V))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Again, nothing to extream in U, S, or V. All values of S are well within the same magnitude.

Intrestingly, I think I may have a small error in my permutation matrix generation, as I need to not swap `np.absolute(pivot_search_space)` to get this PLDU to reconstruct corectly, so perhaps its an index error elseware this corner case touches.
``` python
        abs_pivot_search_space = np.absolute(pivot_search_space)
```

### c.

In [25]:
a3 = np.array([[2, 2, 5],
               [3, 2, 5],
               [1, 1, 5]])
display(Math('A_3=' + bmatrix(a3)))

<IPython.core.display.Math object>

In [26]:
(P, L, D, U) = plud(a3)

eq4 = 'P=' + bmatrix(P) + '\quad ' + \
      'L=' + bmatrix(L) + '\quad ' + \
      'D=' + bmatrix(D) + '\quad ' + \
      'U=' + bmatrix(U)
display(Math(eq4))

A2 = P.dot(L.dot(D.dot(U)))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [34]:
(U, s, V) = np.linalg.svd(a3, full_matrices=False)

# full_matrices=True
# S = np.zeros(U.shape, dtype=complex)
# S[:U.shape[0], :U.shape[1]] = np.diag(s)

# full_matrices=False
S = np.diag(s)

eq4_svd = 'U=' + bmatrix(U) + '\quad \\ ' + \
          'S=' + bmatrix(S)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(V)
display(Math(eq4_svd))

A2 = np.dot(U, np.dot(S, V))
display(Math('A=' + bmatrix(A2)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The reconstruction of A is spot on, and again nothing too small in S.

## 3. 
> Solve the following system of equations $Ax = b$ given the below values for $A$ and $b$. For each system specify if it has zero, one or more solutions. For the systems with zero solutions give the SVD solution. Relate your answers to the SVD decomposition.

### a.

In [27]:
a = np.array([[2, 1, 3],
              [2, 1, 2],
              [5, 5, 5]])
b = np.array([[5],
              [-5],
              [0]])
display(Math('A=' + bmatrix(a) + '\quad ' + 'b=' + bmatrix(b)))

<IPython.core.display.Math object>

By solving for the system of equations, we arive at a well defined solution:

In [28]:
x = np.linalg.solve(a, b)
display(Math('x=' + bmatrix(x)))

<IPython.core.display.Math object>

In [29]:
a_det = np.linalg.det(a)
a_1 = np.linalg.inv(a)
display(Math('Det(A)=' + str(a_det) + '\quad ' + 'A^{-1}=' + bmatrix(a_1)))

<IPython.core.display.Math object>

Here we see that the determinant of A is well behaved, meaing that A is not very singular.

In [30]:
(U, s, V) = np.linalg.svd(a, full_matrices=False)

# full_matrices=True
# S = np.zeros(U.shape, dtype=complex)
# S[:U.shape[0], :U.shape[1]] = np.diag(s)

# full_matrices=False
S = np.diag(s)

eq4_svd = 'U=' + bmatrix(U) + '\quad \\ ' + \
          'S=' + bmatrix(S)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(V)
display(Math(eq4_svd))

x = np.dot(V, np.dot(np.linalg.inv(S), np.dot(np.transpose(U), b)))
display(Math('x=' + bmatrix(x)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

With the SVD solution close to the original, it is more likely that only one solution exist.

### b.

In [107]:
a = np.array([[4, 7, 0],
              [2, 2, -6],
              [1, 2, 1]])
b = np.array([[3],
              [5],
              [1]])
display(Math('A=' + bmatrix(a) + '\quad ' + 'b=' + bmatrix(b)))

<IPython.core.display.Math object>

By solving for the system of equations, we arive at a ill defined solution:

In [108]:
x = np.linalg.solve(a, b)
display(Math('x=' + bmatrix(x)))

<IPython.core.display.Math object>

In [109]:
a_det = np.linalg.det(a)
a_1 = np.linalg.inv(a)
display(Math('Det(A)=' + str(a_det) + '\quad ' + 'A^{-1}=' + bmatrix(a_1)))

<IPython.core.display.Math object>

However, we see that the determinant of A is quite close to 0, meaing that A is almost singular.

In [110]:
(U, s, V) = np.linalg.svd(a, full_matrices=False)

# full_matrices=True
# S = np.zeros(U.shape, dtype=complex)
# S[:U.shape[0], :U.shape[1]] = np.diag(s)

# full_matrices=False
S = np.diag(s)

eq4_svd = 'U=' + bmatrix(U) + '\quad \\ ' + \
          'S=' + bmatrix(S)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(V)
display(Math(eq4_svd))

x = np.dot(V, np.dot(np.linalg.inv(S), np.dot(np.transpose(U), b)))
display(Math('x=' + bmatrix(x)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Here because A is close to singular, and that solution from SVD is quite far from the one found, it is more likly that more solution are possible.

### c.

In [117]:
a = np.array([[4, 7, 0],
              [2, 2, -6],
              [1, 2, 1]])
b = np.array([[18],
              [-12],
              [8]])
display(Math('A=' + bmatrix(a) + '\quad ' + 'b=' + bmatrix(b)))

<IPython.core.display.Math object>

By solving for the system of equations, we arive at a well defined solution:

In [118]:
x = np.linalg.solve(a, b)
display(Math('x=' + bmatrix(x)))

<IPython.core.display.Math object>

In [119]:
a_det = np.linalg.det(a)
a_1 = np.linalg.inv(a)
display(Math('Det(A)=' + str(a_det) + '\quad ' + 'A^{-1}=' + bmatrix(a_1)))

<IPython.core.display.Math object>

However, we see that the determinant of A is quite close to 0, meaing that A is almost singular.

In [114]:
(U, s, V) = np.linalg.svd(a, full_matrices=False)

# full_matrices=True
# S = np.zeros(U.shape, dtype=complex)
# S[:U.shape[0], :U.shape[1]] = np.diag(s)

# full_matrices=False
S = np.diag(s)

eq4_svd = 'U=' + bmatrix(U) + '\quad \\ ' + \
          'S=' + bmatrix(S)
display(Math(eq4_svd))

eq4_svd = 'V=' + bmatrix(V)
display(Math(eq4_svd))

x = np.dot(V, np.dot(np.linalg.inv(S), np.dot(np.transpose(U), b)))
display(Math('x=' + bmatrix(x)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 4. 
> A frequent problem in perception is that you see an object from multiple positions. Consider that you have recorded two datasets $\{a_i\}$ and $\{b_i\}$ with $i = 1 \dots n$ and we assume we have established correspondance between the data points. How could you use SVD or similar techniques to compute the relatively transformation between the datasets $a$ and $b$ assuming that the datasets have many data points and you are expected to use all the data-points.

If we take a look at the homogeneous coordinate transformation that translats and rotates the point from one cordenate frame to the other:

$$
\begin{bmatrix}
    r_{11} & r_{12} & r_{13} & t_{x} \\
    r_{21} & r_{22} & r_{23} & t_{y} \\
    r_{31} & r_{32} & r_{33} & t_{z} \\
    0      & 0      & 0      & 1
\end{bmatrix}
\begin{bmatrix}
    x_{1a} \\
    y_{1a} \\
    z_{1a} \\
    1
\end{bmatrix}
=
\begin{bmatrix}
    x_{1b} \\
    y_{1b} \\
    z_{1b} \\
    1
\end{bmatrix}
,\quad
\begin{bmatrix}
    r_{11} & r_{12} & r_{13} & t_{x} \\
    r_{21} & r_{22} & r_{23} & t_{y} \\
    r_{31} & r_{32} & r_{33} & t_{z} \\
    0      & 0      & 0      & 1
\end{bmatrix}
\begin{bmatrix}
    x_{2a} \\
    y_{2a} \\
    z_{2a} \\
    1
\end{bmatrix}
=
\begin{bmatrix}
    x_{2b} \\
    y_{2b} \\
    z_{2b} \\
    1
\end{bmatrix}
$$ 

We'll see that we can append such transformation together into one large operation.

$$
\begin{bmatrix}
    r_{11} & r_{12} & r_{13} & t_{x} \\
    r_{21} & r_{22} & r_{23} & t_{y} \\
    r_{31} & r_{32} & r_{33} & t_{z} \\
    0      & 0      & 0      & 1
\end{bmatrix}
\begin{bmatrix}
    x_{1a} & \dots & x_{na}\\
    y_{1a} & \dots & y_{na}\\
    z_{1a} & \dots & z_{na}\\
    1      & \dots & 1 
\end{bmatrix}
=
\begin{bmatrix}
    x_{1b} & \dots & x_{nb}\\
    y_{1b} & \dots & y_{nb}\\
    z_{1b} & \dots & z_{nb}\\
    1      & \dots & 1 
\end{bmatrix}
$$

By transposing the equalities above, we can more clearly see the set of linear set of equations we can use to solve for the $R$ and $T$ components of the unknown transformation:

$$
\begin{bmatrix}
    x_{1a} & y_{1a} & z_{1a} & 1     \\
    \vdots & \vdots & \vdots & \vdots \\
    x_{na} & y_{na} & z_{na} & 1     \\
\end{bmatrix}
\begin{bmatrix}
    r_{11} & r_{12} & r_{13} & t_{x} \\
    r_{21} & r_{22} & r_{23} & t_{y} \\
    r_{31} & r_{32} & r_{33} & t_{z} \\
    0      & 0      & 0      & 1
\end{bmatrix}^T
=
\begin{bmatrix}
    x_{1b} & y_{1b} & z_{1b} & 1     \\
    \vdots & \vdots & \vdots & \vdots \\
    x_{nb} & y_{nb} & z_{nb} & 1     \\
\end{bmatrix}
$$

It should be noted that a minimum of 8 associated points forming 8 equations are necessary to solve for the 8 unknown in the rotation and translation of the 3D transformation.

Given the points may be noise, i.e. mensuments made from one references frame may be subject to error, it is perhaps best to Random sample consensus (RANSAC) as an iterative method to estimate the coefficients within the homogeneous transformation matrix. In this way we can ensure that observed data containing outliers from mesurment noise are to be accorded no influence on eventual estimate.